In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import warnings
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")
np.random.seed(42)

In [2]:
df = pd.read_csv('train.csv')
y = df.target

df.drop(['ID', 'target'], axis=1, inplace=True)
df.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,NaN,1.312910,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,NaN,1.050328,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN


In [3]:
string_type = []
for column in df.columns:
    if type(df[column].values[0]) == str:
        string_type.append(column)
string_type.append('v113')
        
df[string_type] = df[string_type].fillna('zero')

df.fillna(-9999, inplace=True)
cat_features_ids = np.where(df.apply(pd.Series.nunique) < 10000)[0].tolist()

In [4]:
df.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,0.503281,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,-9999.000000,-9999.000000,C,-9999.000000,9.191265,-9999.000000,-9999.000000,2.301630,-9999.000000,1.312910,...,-9999.000000,-9999.000000,0.598896,AF,-9999.000000,-9999.000000,1.957825,0,-9999.000000,-9999.000000
2,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,0.765864,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,6.542669,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,-9999.000000,-9999.000000,C,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,1.050328,...,-9999.000000,-9999.000000,-9999.000000,Z,-9999.000000,-9999.000000,-9999.000000,0,-9999.000000,-9999.000000


In [5]:
le = preprocessing.LabelEncoder()
for column in string_type:
    df[column] = le.fit_transform(df[column])

In [6]:
train, test, y_train, y_test = train_test_split(df, y, test_size = 0.2)

In [7]:
lr = LogisticRegression()
lr.fit(train, y_train)
y_pred = lr.predict(test)
print('Accuracy score baseline:', accuracy_score(y_test, y_pred))

Accuracy score baseline: 0.7616881696916685


In [8]:
folds = pd.DataFrame(list(range(len(train))))
folds[0] = folds.values % 3
folds.rename(columns={0:'fold'},inplace=True)

## Catboost

In [9]:
dict_OOF_predict_cbst = {}
dict_test_predict_cbst = {}
for fold in [0,1,2]:

    
    clf = CatBoostClassifier(learning_rate=0.1, iterations=500, random_seed=42, logging_level='Silent')
    clf.fit(train.values[folds.fold != fold], y_train.values[folds.fold != fold], cat_features=cat_features_ids)
    
    predicts = clf.predict_proba(test)[:,1]
    dict_test_predict_cbst[fold] = predicts

    predicts_OOF = clf.predict_proba(train.values[folds.fold == fold])[:,1]
    dict_OOF_predict_cbst[fold] = predicts_OOF
    del clf
    


In [20]:
    clf = CatBoostClassifier(learning_rate=0.1, iterations=500, random_seed=42, logging_level='Silent')
    clf.fit()

In [10]:
OOF_X_cbst = np.zeros_like(y_train)
OOF_X_cbst = pd.DataFrame(OOF_X_cbst)
for fold in dict_OOF_predict_cbst.keys():
    OOF_X_cbst[folds.fold == fold] = dict_OOF_predict_cbst[fold].reshape((dict_OOF_predict_cbst[fold].shape[0],1))
print('OOF Catboost accuracy',accuracy_score(y_train,np.round(OOF_X_cbst)))

OOF Catboost accuracy 0.7817748425472358


In [11]:
f_0,f_1,f_2 = dict_test_predict_cbst.keys()
sub_cbst = dict_test_predict_cbst[f_0] + dict_test_predict_cbst[f_1] + dict_test_predict_cbst[f_2]
sub_cbst/=3
print('test Catboost accuracy',accuracy_score(y_test,np.round(sub_cbst)))

test Catboost accuracy 0.7865296304395364


## KNN 2

In [12]:

dict_OOF_predict_knn_2 = {}
dict_test_predict_knn_2  = {}
for fold in [0,1,2]:

    
    clf = KNeighborsClassifier(n_neighbors = 2,weights='distance',n_jobs=32)
    clf.fit(train.values[folds.fold != fold], y_train.values[folds.fold != fold])
    
    predicts = clf.predict_proba(test)[:,1]
    dict_test_predict_knn_2[fold] = predicts

    predicts_OOF = clf.predict_proba(train.values[folds.fold == fold])[:,1]
    dict_OOF_predict_knn_2[fold] = predicts_OOF
    del clf
    
OOF_X_knn_2 = np.zeros_like(y_train)
OOF_X_knn_2 = pd.DataFrame(OOF_X_knn_2)
for fold in dict_OOF_predict_knn_2.keys():
    OOF_X_knn_2[folds.fold == fold] = dict_OOF_predict_knn_2[fold].reshape((dict_OOF_predict_knn_2[fold].shape[0],1))
print('OOF kNN 2 accuracy',accuracy_score(y_train,np.round(OOF_X_knn_2)))

f_0,f_1,f_2 = dict_test_predict_knn_2.keys()
sub_knn_2 = dict_test_predict_knn_2[f_0] + dict_test_predict_knn_2[f_1] + dict_test_predict_knn_2[f_2]
sub_knn_2/=3
print('test kNN 2 accuracy',accuracy_score(y_test,np.round(sub_knn_2)))

OOF kNN 2 accuracy 0.6638820853743876
test kNN 2 accuracy 0.7161163350098404


## KNN 4

In [13]:

dict_OOF_predict_knn_4 = {}
dict_test_predict_knn_4  = {}
for fold in [0,1,2]:

    
    clf = KNeighborsClassifier(n_neighbors= 4,weights='distance',n_jobs=32)
    clf.fit(train.values[folds.fold != fold], y_train.values[folds.fold != fold])
    
    predicts = clf.predict_proba(test)[:,1]
    dict_test_predict_knn_4[fold] = predicts

    predicts_OOF = clf.predict_proba(train.values[folds.fold == fold])[:,1]
    dict_OOF_predict_knn_4[fold] = predicts_OOF
    del clf
    
OOF_X_knn_4 = np.zeros_like(y_train)
OOF_X_knn_4 = pd.DataFrame(OOF_X_knn_4)
for fold in dict_OOF_predict_knn_4.keys():
    OOF_X_knn_4[folds.fold == fold] = dict_OOF_predict_knn_4[fold].reshape((dict_OOF_predict_knn_4[fold].shape[0],1))
print('OOF kNN 4 accuracy',accuracy_score(y_train,np.round(OOF_X_knn_4)))

f_0,f_1,f_2 = dict_test_predict_knn_4.keys()
sub_knn_4 = dict_test_predict_knn_4[f_0] + dict_test_predict_knn_4[f_1] + dict_test_predict_knn_4[f_2]
sub_knn_4/=3
print('test kNN 4 accuracy',accuracy_score(y_test,np.round(sub_knn_4)))

OOF kNN 4 accuracy 0.7044808432470259
test kNN 4 accuracy 0.7345287557402143


## KNN 8

In [14]:

dict_OOF_predict_knn_8 = {}
dict_test_predict_knn_8  = {}
for fold in [0,1,2]:

    
    clf = KNeighborsClassifier(n_neighbors = 8,weights='distance',n_jobs=32)
    clf.fit(train.values[folds.fold != fold], y_train.values[folds.fold != fold])
    
    predicts = clf.predict_proba(test)[:,1]
    dict_test_predict_knn_8[fold] = predicts

    predicts_OOF = clf.predict_proba(train.values[folds.fold == fold])[:,1]
    dict_OOF_predict_knn_8[fold] = predicts_OOF
    del clf
    
OOF_X_knn_8 = np.zeros_like(y_train)
OOF_X_knn_8 = pd.DataFrame(OOF_X_knn_8)
for fold in dict_OOF_predict_knn_8.keys():
    OOF_X_knn_8[folds.fold == fold] = dict_OOF_predict_knn_8[fold].reshape((dict_OOF_predict_knn_8[fold].shape[0],1))
print('OOF kNN 8 accuracy',accuracy_score(y_train,np.round(OOF_X_knn_8)))

f_0,f_1,f_2 = dict_test_predict_knn_8.keys()
sub_knn_8 = dict_test_predict_knn_8[f_0] + dict_test_predict_knn_8[f_1] + dict_test_predict_knn_8[f_2]
sub_knn_8/=3
print('test kNN 8 accuracy',accuracy_score(y_test,np.round(sub_knn_8)))

OOF kNN 8 accuracy 0.7372397655703289
test kNN 8 accuracy 0.7567461185217581


## KNN 16

In [15]:

dict_OOF_predict_knn_16 = {}
dict_test_predict_knn_16  = {}
for fold in [0,1,2]:

    
    clf = KNeighborsClassifier(n_neighbors = 16,weights='distance',n_jobs=32)
    clf.fit(train.values[folds.fold != fold], y_train.values[folds.fold != fold])
    
    predicts = clf.predict_proba(test)[:,1]
    dict_test_predict_knn_16[fold] = predicts

    predicts_OOF = clf.predict_proba(train.values[folds.fold == fold])[:,1]
    dict_OOF_predict_knn_16[fold] = predicts_OOF
    del clf
    
OOF_X_knn_16 = np.zeros_like(y_train)
OOF_X_knn_16 = pd.DataFrame(OOF_X_knn_16)
for fold in dict_OOF_predict_knn_16.keys():
    OOF_X_knn_16[folds.fold == fold] = dict_OOF_predict_knn_16[fold].reshape((dict_OOF_predict_knn_16[fold].shape[0],1))
print('OOF kNN 16 accuracy',accuracy_score(y_train,np.round(OOF_X_knn_16)))


f_0,f_1,f_2 = dict_test_predict_knn_16.keys()
sub_knn_16 = dict_test_predict_knn_16[f_0] + dict_test_predict_knn_16[f_1] + dict_test_predict_knn_16[f_2]
sub_knn_16/=3
print('test kNN 16 accuracy',accuracy_score(y_test,np.round(sub_knn_16)))

OOF kNN 16 accuracy 0.75659333449965
test kNN 16 accuracy 0.7642685326918872


## Stacking

In [16]:
stacked_data_set_train = pd.DataFrame(OOF_X_cbst)
stacked_data_set_test = pd.DataFrame(sub_cbst)

stacked_data_set_train[1] = OOF_X_knn_2
stacked_data_set_test[1] = sub_knn_2

stacked_data_set_train[2] = OOF_X_knn_4
stacked_data_set_test[2] = sub_knn_4 

stacked_data_set_train[3] = OOF_X_knn_8
stacked_data_set_test[3] = sub_knn_8

stacked_data_set_train[4] = OOF_X_knn_16
stacked_data_set_test[4] = sub_knn_16 


In [17]:
lr = LogisticRegression()
lr.fit(stacked_data_set_train, y_train)
y_pred = lr.predict(stacked_data_set_test)
print('Accuracy score baseline:', accuracy_score(y_test, y_pred))

Accuracy score baseline: 0.787797944456593


In [18]:
original_score = 0.7865296304395364
best_score = 0.787797944456593
improvement = np.abs(np.round(100*(original_score - best_score)/original_score,2))
print('overall improvement with stacking is {} %'.format(improvement))

overall improvement with stacking is 0.16 %


In [19]:
print('additional value is {} samples'.format(df.shape[0] * 0.16 / 100))

additional value is 182.9136 samples
